# Surprise homework

In [187]:
import io 
from collections import defaultdict
import pandas as pd
from surprise import Dataset
from surprise import SVD
from surprise import NormalPredictor
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

### Functions

In [188]:
# используйте полезные функции из FAQ

def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=5, threshold=3.52):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

### Load data

In [189]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25)
ratings = trainset.all_ratings()
item_path = '~/.surprise_data/ml-100k/ml-100k/u.item'
item_df = pd.read_csv(item_path, sep='|',encoding='ISO-8859-1', header = None)

### Define algorithms

In [190]:
# внимательно изучите документацию по метрикам и алгоритмам

### Select algorithm

In [191]:
n = 30
variant = '33'
accurancy = 3
n_cv = 5

SVD алгоритм

In [192]:
algoSVD = SVD()
algoSVD.fit(trainset)
predictionsSVD = algoSVD.test(testset)
rmseSVD = accuracy.rmse(predictionsSVD)

RMSE: 0.9373


In [209]:
cvSVD = cross_validate(algoSVD, data, measures=['RMSE'], cv=n_cv, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9397  0.9312  0.9296  0.9303  0.9384  0.9338  0.0043  
Fit time          4.05    4.05    4.09    4.13    4.12    4.09    0.04    
Test time         0.12    0.12    0.11    0.12    0.12    0.12    0.00    


User-based коллаборативная фильтрация, метод kNN, k = 30, метрика Mean
Squared Difference 

In [193]:
algoMSD = KNNWithMeans(k=n)
algoMSD.fit(trainset)
predictionsMSD = algoMSD.test(testset)
rmseMSD = accuracy.rmse(predictionsMSD)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9523


In [210]:
cvMSD = cross_validate(algoMSD, data, measures=['RMSE'], cv=n_cv, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9537  0.9447  0.9540  0.9593  0.9513  0.9526  0.0047  
Fit time          0.32    0.33    0.33    0.31    0.33    0.32    0.01    
Test time         2.83    2.66    3.03    2.76    2.82    2.82    0.12    


User-based коллаборативная фильтрация, метод kNN, k = 30, метрика
косинуса

In [194]:
algoCos = KNNWithMeans(k=n, sim_options={'name': 'cosine'})
algoCos.fit(trainset)
predictionsCos = algoCos.test(testset)
rmseCos = accuracy.rmse(predictionsCos)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9581


In [212]:
cvCos = cross_validate(algoCos, data, measures=['RMSE'], cv=n_cv, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9609  0.9709  0.9537  0.9497  0.9562  0.9583  0.0073  
Fit time          0.81    0.80    0.84    0.79    0.76    0.80    0.03    
Test time         2.83    2.69    2.98    2.44    2.42    2.67    0.22    


User-based коллаборативная фильтрация, метод kNN, k = 30, метрика
корреляция Пирсона

In [195]:
algoPsn = KNNWithMeans(k=n, sim_options={'name': 'pearson_baseline'})
algoPsn.fit(trainset)
predictionsPsn = algoPsn.test(testset)
rmsePsn = accuracy.rmse(predictionsPsn)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9395


In [213]:
cvPsn = cross_validate(algoPsn, data, measures=['RMSE'], cv=n_cv, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9323  0.9414  0.9330  0.9433  0.9363  0.9372  0.0044  
Fit time          0.93    0.96    0.93    0.95    1.01    0.96    0.03    
Test time         2.60    2.47    2.39    2.57    2.69    2.55    0.10    


Прогнозирование случайного рейтинга на основе распределения всех рейтингов в
наборе

In [196]:
algoNorm = NormalPredictor()
algoNorm.fit(trainset)
predictionsNorm = algoNorm.test(testset)
rmseNorm = accuracy.rmse(predictionsNorm)

RMSE: 1.5250


In [211]:
cvNorm = cross_validate(algoNorm, data, measures=['RMSE'], cv=n_cv, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5156  1.5203  1.5066  1.5117  1.5326  1.5174  0.0089  
Fit time          0.14    0.15    0.14    0.14    0.14    0.14    0.00    
Test time         0.28    0.11    0.12    0.27    0.11    0.18    0.08    


In [214]:
predictions = predictionsSVD

### Calculate precision@k and recall@k

In [215]:
precisions, recalls = precision_recall_at_k(predictions)

In [216]:
precisionak = round(sum(precision for precision in precisions.values()) / len(precisions), accurancy)
precisionak

0.739

In [217]:
recallak = round(sum(recall for recall in recalls.values()) / len(recalls), accurancy)
recallak

0.366

### Predict

In [218]:
testset = trainset.build_anti_testset()
top = get_top_n(predictions)

movies = list(filter(lambda x: x[0] == variant, top.items()))[0][1]
movies

[('307', 3.960695412029825),
 ('288', 3.8251009430555185),
 ('348', 3.662202051392625),
 ('880', 3.6069686547833006),
 ('751', 3.4945827988416234)]

In [219]:
def read_items(movies):
    description = []
    for movie in movies:
        row = item_df.iloc[int(movie[0])]
        description.append([movie[0], (row[1], row[2]), round(movie[1], accurancy)])
    return description
result = read_items(movies)
result

[['307', ('FairyTale: A True Story (1997)', '01-Jan-1997'), 3.961],
 ['288', ('Evita (1996)', '25-Dec-1996'), 3.825],
 ['348', ('Hard Rain (1998)', '16-Jan-1998'), 3.662],
 ['880', ('Money Talks (1997)', '22-Aug-1997'), 3.607],
 ['751', ('Replacement Killers, The (1998)', '06-Feb-1998'), 3.495]]

In [220]:
file = open('hw22.txt', 'w')
print('User '+ variant)
file.write('User '+ variant + '\n')
for movie in result:
    line = str(movie[0]) + ' ' + str(movie[1]) + ' ' + str(movie[2])
    print(line)
    file.write(line + '\n')
file.close()

User 33
307 ('FairyTale: A True Story (1997)', '01-Jan-1997') 3.961
288 ('Evita (1996)', '25-Dec-1996') 3.825
348 ('Hard Rain (1998)', '16-Jan-1998') 3.662
880 ('Money Talks (1997)', '22-Aug-1997') 3.607
751 ('Replacement Killers, The (1998)', '06-Feb-1998') 3.495
